# Sprint Modulo 4

Karen Tapia

Se importan librerias para la ejecucion del algoritmo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np


house_path = "/content/drive/MyDrive/house.csv"
#house_path = "house.csv"

#archivos procesados en pandas
df_house = pd.read_csv(house_path)

df_house.head(10) #despliega un encabezado de 10 filas


In [ ]:
#eliminacion de las columnas indicadas
columns_eliminar = ['Alley', 'FireplaceQu','PoolQC', 'Fence', 'MiscFeature', 'LotFrontage']
df_house.drop(columns=columns_eliminar, inplace=True)
print("Columnas eliminadas")
df_house.info()


In [ ]:
#limpieza de datos revicion de valores NaN
#contabilizar los NaN
nan_counts = df_house.isna().sum()
#filtrar solo las columnas que tiene valores NaN
nan_columns = nan_counts[nan_counts > 0]
nan_columns


In [ ]:
#limpieza de datos para las variables numericas y categoricas
#para las variables numericas, asignamos la mediana
num_columns_with_nan=['MasVnrArea', 'GarageYrBlt']
for column in num_columns_with_nan:
    df_house[column].fillna(df_house[column].median(), inplace=True)

  #para las variables categoricas, asignamos el valor "desconocido"
cat_columns_with_nan = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                          'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType',
                          'GarageFinish', 'GarageQual','GarageCond']
for column in cat_columns_with_nan:
    df_house[column].fillna('Desconocido', inplace=True)
  #revisando los valores NaN
nan_counts = df_house.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
nan_columns

In [ ]:
#resumen estadistico de las variables numericas
summary_statistics = df_house.describe()
summary_statistics.head()


In [ ]:
#configuracion de los graficos para totalBsmtSF, SalePrice y GrLivArea en una sola salida
fig, axes = plt.subplots(1,3,figsize=(18,5))
#TotalBsmtSF
sns.histplot(df_house['TotalBsmtSF'], kde=True, ax=axes[0])
axes[0].set_title('Total Basement Area (sq feet)')
#SalePrice
sns.histplot(df_house['SalePrice'], kde=True, ax=axes[1])
axes[1].set_title('Sale Price')
#GrLivArea
sns.histplot(df_house['GrLivArea'], kde=True, ax=axes[2])
axes[2].set_title('Above Groud Living Area (sq feat)')
plt.tight_layout()
plt.show()


In [ ]:
#Crear diagramas de boxplot para TotalBsmtSF, SalePrice y GrLivArea para identificar outliers
fig, axes=plt.subplots(1,3,figsize=(18,5))
#TotalBsmtSF
sns.boxplot(x=df_house['TotalBsmtSF'], ax=axes[0])
axes[0].set_title('TotalBsmtSF Area (sq feet)')
#SalePrice
sns.boxplot(x=df_house['SalePrice'],ax=axes[1])
axes[1].set_title('SalePrice')
#GrLivArea
sns.boxplot(x=df_house['GrLivArea'], ax=axes[2])
axes[2].set_title('Above Ground Living Area (sq feet)')

plt.tight_layout()
plt.show()

In [ ]:
#eliminacion de outliers
#funcion para eliminar outliers basados en IQR
def remove_outliers(df, column):
    Q1= df[column].quantile(0.25)
    Q3= df[column].quantile(0.75)
    IQR = Q3-Q1
    lower_bound = Q1 - 1.5 *IQR
    upper_bound = Q3+1.5*IQR
    return df[(df[column] >= lower_bound)&(df[column]<= upper_bound)]

#eliminar outliers de las tres variables
df_cleaned = remove_outliers(df_house, 'TotalBsmtSF')
#estos dos codigos ya incorporan df_house, por eso los elimina desde df_cleaned
df_cleaned= remove_outliers(df_cleaned,'SalePrice')
df_cleaned= remove_outliers(df_cleaned, 'GrLivArea')
#crear nuevamente diagramas de boxplot para las variables despues de eliminar outliers
fig, axes = plt.subplots(1,3,figsize=(18,5))
#TotalBsmtSF
sns.boxplot(x=df_cleaned['TotalBsmtSF'],ax=axes[0])
axes[0].set_title('Total basement Area (sq feet) - Cleaned')
#salePrice
sns.boxplot(x=df_cleaned['SalePrice'], ax=axes[1])
axes[1].set_title('Sale Price - cleaned')
#GrLivArea
sns.boxplot(x=df_cleaned['GrLivArea'], ax=axes[2])
axes[2].set_title('above ground living area (sq feet) - Cleaned')
plt.tight_layout()
plt.show

In [ ]:
#Grafico de distribucion de las variables
fig, axes = plt.subplots(1,3, figsize=(18,5))
#TotalBsmtSF
sns.histplot(df_cleaned['TotalBsmtSF'], kde=True, ax=axes[0])
axes[0].set_title('Total Basement Area (sq feet)')
#SalePrice
sns.histplot(df_cleaned['SalePrice'], kde=True, ax=axes[1])
axes[1].set_title('Sale Price')
#GrLivArea
sns.histplot(df_cleaned['GrLivArea'], kde=True, ax=axes[2])
axes[2].set_title('Above Ground Living Area (sq feet)')
plt.tight_layout()
plt.show()


In [ ]:
from scipy import stats
import numpy as np #se importa libreria numpy
#definir el nivel de confianza
confidence_level=0.95
#calcular el intervalo de confianza para el precio de venta promedio
sale_price_mean = df_cleaned['SalePrice'].mean()
sale_price_std = df_cleaned['SalePrice'].std()
n=len(df_cleaned['SalePrice'])
z_critical = stats.norm.ppf((1+confidence_level)/2) #Z-score para el nivelde confianza del 95%
#margen de error
margin_of_error = z_critical *(sale_price_std/np.sqrt(n))
#intervalo de confianza
confidence_interval = (sale_price_mean - margin_of_error, sale_price_mean + margin_of_error)
#umbral de accesibilidad en terminos monetarios (hipotetico)
accessibility_threshold = 150000
#realizar la prueba de hipotesis

t_stat, p_value = stats.ttest_1samp(df_cleaned['SalePrice'], accessibility_threshold)
#resultados
print(f"intervalo de confianza: {confidence_interval}")
print(f"valor t {t_stat}")
print(f"valor t {p_value}")


El resultado de la prueba de hipótesis nos dice que el precio de venta promedio de las casas es estadísticamente significativo y diferente del umbral de de accesibilidad de $150.000 que habíamos considerado. Con valor p muy por debajo  del nivel de significancia estándar de 0.05,  rechazamos la hipótesis nula de que el precio de venta promedio es igual a

$150.000 esto indica que, en promedio, las casas en este mercado son más caras que nuestro umbral definido para la accesibilidad.
En base a las conclusiones obtenidas en el presente análisis, podemos señalar que para un adulto joven recién egresado buscando comprar su primera casa, el mercado actual puede presentar desafíos en términos de precios. Las casas, en promedio están por encima del umbral de accesibilidad preestablecido, lo que sugiere necesidad de una planificación financiera cuidadosa y la exploración de opciones de vivienda que ofrezcan un buen equilibrio entre precio, ubicación y características deseables.
Para elaborar un análisis más preciso y realista, resulta esencial incorporar  un espectro más amplio de variables, así como llevar a cabo un  exhaustivo estudio de mercado que permita determinar un valor adecuado para el umbral de accesibilidad vigente. Un caso ilustrativo de esto se observa en chile. Donde los eventos anteriores y posteriores a la pandemia han provocado una escalada en los precios del sector inmobiliario. Este fenómeno se ha complicado de manera creciente la adquisición de vivienda propia para los profesionales jóvenes, evidenciando la dinámica y volatilidad del mercado. En este contexto, comprender las fluctuaciones tendencias actuales se vuelve fundamental para ajustar nuestras estimaciones y recomendaciones, asegurando que reflejen las condiciones del mercado en tiempo real y proporcionen una guía valiosa para la toma de decisiones informadas en el ámbito de la vivienda.

